# Lending Clug Case Study: Credit Risk
description...

## Imports & Settings

In [1]:
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

pd.options.plotting.backend = "plotly"

In [2]:
# for better visibility, we will display the whole dataframe and show floats with 2 decimal places
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# set the renderer to disply plots in VSCode
pio.renderers.default = "notebook_connected"

## First Look

In [3]:
df = pd.read_csv('loan.csv')
df.set_index('id', inplace=True)
df.head(5)

/var/folders/jb/y9g66hr95b90jzyhlqzhvcw40000gn/T/ipykernel_32692/2115001733.py:1: DtypeWarning:

Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.



,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.00,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.16,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077430,1314167,2500,2500,2500.00,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1077175,1313524,2400,2400,2400.00,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.67,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1076863,1277178,10000,10000,10000.00,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.00,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.00,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.89,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,N

## Data Clean Up

While importing, Pandas already warns us that there are columns with mixed data types. 
> 'DtypeWarning: Columns (47) have mixed types.'

For the analysis we need our data to be correctly formatted (preferably numeric).

### Empty & Monotonous Columns
If the data contains completely empty columns we can drop them as they don't contain any information to analyze.

In [4]:
empty_cols = [col for col in df.columns if df[col].isnull().all()]
print(len(empty_cols), 'empty columns found')
print('Empty columns:', empty_cols)

df.drop(empty_cols, axis=1, inplace=True)

54 empty columns found
Empty columns: ['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort', 't

In [5]:
# reviewing distribution of null values in columns

col_null_values = df.isnull().sum()
col_null_values.hist(nbins=20)

Columns that contain nothing but one value don't add any information and can be droped. We also look at columns with 2 values to see if they can be dropped.

In [6]:
cols_with_1_value = [col for col in df.columns if df[col].nunique() == 1]
df.drop(cols_with_1_value, axis=1, inplace=True)


cols_with_2_values = [col for col in df.columns if df[col].nunique() == 2]
for col in cols_with_2_values:
    print(col, df[col].unique())

term [' 36 months' ' 60 months']
next_pymnt_d [nan 'Jun-16' 'Jul-16']


In [7]:
df.next_pymnt_d.value_counts()

Jun-16    1125
Jul-16      15
Name: next_pymnt_d, dtype: int64

Just the 'term' (duration of the loan) (36 or 60 months) and the 'next_pymnt_d' (June or July 2016 if any) have only 2 different values. 
<br>Duration might be a factor associated with credit risk, so we leave that column in.
<br>We assume that the cut-off date was in or before June 2016, so that the entries in 'next_pymnt_d' were in the future when the data was taken. This would explain why there are so few and it fits with the time frame of the project description 
(2011 + 60 months = 2017). We don't know if these payments happened but might need the information and therefor leave the column in.

In [8]:
# Since we are interested in finding predictors for default risk a.k.a. the 'loan_status' column, we can drop the currently active contracts as we don't know their outcome.

df = df[df['loan_status'] != 'Current']

# We can also check for null values in the loan status column.
df['loan_status'].isnull().sum()

0

### Units

We can convert units that are spelled out in each cell to numeric values

In [9]:
# converting all columns with units in in values to numbers
df['term'] = df['term'].str.replace(' months', '').astype(int)
df.rename(columns={'term': 'term_in_months'}, inplace=True)

df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
df['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

In [10]:
df.emp_length.unique()

array(['10+ years', '< 1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '1 year', '6 years', '2 years', '7 years',
       nan], dtype=object)

We see above that the employment length has 3 vaules (<1,1 and 10+) that don't end in 'years'. We can convert these.


In [11]:
df['emp_length'] = df['emp_length'].replace('10+ years', 10).astype(str)
df['emp_length'] = df['emp_length'].replace('< 1 year', 0.5).astype(str)
df['emp_length'] = df['emp_length'].replace('1 year',     1).astype(str)

df['emp_length'] = df['emp_length'].str.replace(' years', '').astype(float)


### Dates

In [12]:
# Looking at the table we can see the following columns contain dates
cols_with_date = [
                  'issue_d', 'earliest_cr_line',
                  'last_pymnt_d', 'next_pymnt_d', 
                  'last_credit_pull_d'
                  ]

for col in cols_with_date:
    df[col] = pd.to_datetime(df[col], format='%b-%y')


@Justin: do you know a way of detecting dates other than 'Looking at the table' 😅

@Krilecy: Unfortunately not! We _could_ use datetime to identify if a column was of a certain object type, but I think we'd still have to look at the date format to see how we need to re-format it 😢

### Description Column

The description column ('desc') contains unstructured data. Let's see if we can extract usefull information from it.

In [13]:
print(len(df.desc.unique()), 'unique descriptions')

print('Examples:')
for desc in df.sample(5).desc:
    print('-', str(desc).strip())


25804 unique descriptions
Examples:
- Borrower added on 03/25/10 > I have a final bill due to my lawyer to cover legal bills incurrerd during a rather long and contentious custody battle.  This loan will cover the amount outstanding to my attorney - $7000 - as well as pay-off a current Lending Club loan.  A nice consolidation and easy payment process will be the result.  Thank you for the opportunity!<br/>
- I'm looking to consolidate a few credit cards before I get married.  Also looking to buy parts for my truck; losing oil pressure and need to keep it on the road.
- I will pay off and cancel two high interest credit cards
- Borrower added on 03/31/11 > I'm working on paying off some credit card debt before going to graduate school. This loan will be make it possible to consolidate my debt and work on paying it down faster. The faster I get this debt paid off, the faster I can move on to the next step in my life.<br/> Borrower added on 03/31/11 > I work full-time and have developed a

The descriptions often contain the date when the borrower was added and their usecase. Although the usecase might be interesting for further analysis we only extract the date so we can use it in the bivariate analysis later.

In [14]:
# extract the date from the description if the description starts with 'Borrower added on'
def extract_added_date(desc):
    desc = str(desc).strip()
    if desc.startswith('Borrower added on'):
        return desc.split(' >')[0][-8:]
    else:
        return None

# insert the new column next to the description column and convert it to datetime
df.insert(df.columns.get_loc('desc') + 1,'borrower_added_d',df.desc.apply(extract_added_date))
df['borrower_added_d'] = pd.to_datetime(df['borrower_added_d'], format='%m/%d/%y', errors='coerce')

In [15]:
# The web addresses of the URL column are inaccessible and therefore beyond the scope of this assignment.
df.drop(['url'], axis=1, inplace=True)

### Grades

In [16]:
# Converting the sub_grade scale to categorical numerical values

df['grade_num'] = df.sub_grade.astype("category").cat.codes
df['grade_num'].value_counts()

3     2873
7     2825
4     2715
9     2615
8     2437
10    2055
6     2001
11    1931
2     1810
5     1797
1     1508
12    1488
16    1286
13    1206
14    1154
0     1139
17    1116
15     931
18     918
19     834
20     722
21     614
22     516
23     424
24     387
25     305
26     233
27     174
28     151
29     113
30      94
31      77
33      54
32      45
34      29
Name: grade_num, dtype: int64

## Data Dictionary

In [17]:
# For easy access we can create a dataframe with the descriptions from the accompanying Data_Dictionary.xlsx

data_dict = pd.read_excel('Data_Dictionary.xlsx', index_col=0, header=None).iloc[:, 0:1].transpose()
data_dict

,LoanStatNew,acc_now_delinq,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,application_type,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,desc,dti,dti_joint,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,id,il_util,initial_list_status,inq_fi,inq_last_12m,inq_last_6mths,installment,int_rate,issue_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,max_bal_bc,member_id,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,next_pymnt_d,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_acc_6m,open_il_12m,open_il_24m,open_il_6m,open_rv_12m,open_rv_24m,out_prncp,out_prncp_inv,pct_tl_nvr_dlq,percent_bc_gt_75,policy_code,pub_rec,pub_rec_bankruptcies,purpose,pymnt_plan,recoveries,revol_bal,revol_util,sub_grade,tax_liens,term,title,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,url,verification_status,verified_status_joint,zip_code,NaN,NaN
1,Description,The number of accounts on which the borrower i...,Number of trades opened in past 24 months.,The state provided by the borrower in the loan...,Balance to credit limit on all trades,The self-reported annual income provided by th...,The combined self-reported annual income provi...,Indicates whether the loan is an individual ap...,Average current balance of all accounts,Total open to buy on revolving bankcards.,Ratio of total current balance to high credit/...,Number of charge-offs within 12 months,post charge off collection fee,Number of collections in 12 months excluding m...,The number of 30+ days past-due incidences of ...,The past-due amount owed for the accounts on w...,Loan description provided by the borrower,A ratio calculated using the borrower’s total ...,A ratio calculated using the co-borrowers' tot...,The month the borrower's earliest reported cre...,Employment length in years. Possible values ar...,The job title supplied by the Borrower when ap...,The upper boundary range the borrower’s FICO a...,The lower boundary range the borrower’s FICO a...,The total amount committed to that loan at tha...,The total amount committed by investors for th...,LC assigned loan grade,The home ownership status provided by the borr...,A unique LC assigned ID for the loan listing.,Ratio of total current balance to high credit/...,The initial listing status of the loan. Possib...,Number of personal finance inquiries,Number of credit inquiries in past 12 months,The number of inquiries in past 6 months (excl...,The monthly payment owed by the borrower if th...,Interest Rate on the loan,The month which the loan was funded,The most recent month LC pulled credit for thi...,The upper boundary range the borrower’s last F...,The lower boundary range the borrower’s last F...,Last total payment amount received,Last month payment was received,The listed amount of the loan applied for by t...,Current status of the loan,Maximum current balance owed on all revolving ...,A unique LC assigned Id for the borrower member.,Months since oldest bank installment account o...,Months since oldest revolving account opened,Months since most recent revolving account opened,Months since most recent account opened,Number of mortgage accounts.,The number of months since the borrower's last...,Months since most recent 

@Justin we might want to change the type to a Python dict when using it later. I'll leave it as is for now. Let's see what we do with it. 

## Overview

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38577 entries, 1077501 to 87023
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   member_id                38577 non-null  int64         
 1   loan_amnt                38577 non-null  int64         
 2   funded_amnt              38577 non-null  int64         
 3   funded_amnt_inv          38577 non-null  float64       
 4   term_in_months           38577 non-null  int64         
 5   int_rate                 38577 non-null  float64       
 6   installment              38577 non-null  float64       
 7   grade                    38577 non-null  object        
 8   sub_grade                38577 non-null  object        
 9   emp_title                36191 non-null  object        
 10  emp_length               37544 non-null  float64       
 11  home_ownership           38577 non-null  object        
 12  annual_inc               3

In [19]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term_in_months,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,grade_num
count,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,37544.00,38577.00,38577.00,38577.00,38577.00,13672.00,2740.00,38577.00,38577.00,38577.00,38527.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,38577.00,37880.00,38577.00
mean,842284.34,11047.03,10784.06,10222.48,41.90,11.93,322.47,5.00,68777.97,13.27,0.15,0.87,35.88,69.26,9.28,0.06,13289.49,48.70,22.05,0.00,0.00,11866.97,11271.20,9648.32,2119.24,1.37,98.04,12.77,2746.24,0.04,9.82
std,264451.93,7348.44,7090.31,7022.72,10.33,3.69,208.64,3.48,64218.68,6.67,0.49,1.07,22.03,43.99,4.40,0.24,15866.49,28.36,11.43,0.00,0.00,8809.86,8700.85,7009.38,2389.50,7.32,698.65,150.84,4494.65,0.20,6.77
min,70699.00,500.00,500.00,0.00,36.00,5.42,15.69,0.50,4000.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,661131.00,5300.00,5200.00,5000.00,36.00,8.94,165.74,2.00,40000.00,8.13,0.00,0.00,18.00,19.75,6.00,0.00,3650.00,25.20,13.00,0.00,0.00,5513.50,5019.59,4500.00,643.99,0.00,0.00,0.00,217.42,0.00,4.00
50%,839292.00,9600.00,9550.00,8733.44,36.00,11.71,277.86,4.00,58868.00,13.37,0.00,1.00,34.00,90.00,9.00,0.00,8762.00,49.10,20.00,0.00,0.00,9674.05,9066.66,8000.00,1300.45,0.00,0.00,0.00,568.26,0.00,9.00
75%,1037336.00,15000.00,15000.00,14000.00,36.00,14.38,425.55,9.00,82000.00,18.56,0.00,1.00,52.00,104.00,12.00,0.00,16912.00,72.30,29.00,0.00,0.00,16136.95,15329.78,13175.00,2671.98,0.00,0.00,0.00,3447.25,0.00,14.00
max,1314167.00,35000.00,35000.00,35000.00,60.00,24.40,1305.19,10.00,6000000.00,29.99,11.00,8.00,120.00,129.00,44.00,4.00,149588.00,99.90,90.00,0.00,0.00,58563.68,58563.68,35000.02,23563.68,180.20,29623.35,7002.19,36115.20,2.00,34.00


Now that we've done some cleaning up of the data, we can begin to explore and understand different feature at length and its surface level impact to loan _status. We'll start with features we feel could be valuable indicators of correlation to loan_status.

## Feature Analysis

### Function Definitions

In [20]:
def plot_distribution(column_name):

    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(
        go.Violin(x=df[column_name], name="Violin plot"),
        row=1, col=1
        )
    
    fig.add_trace(
        go.Box(x=df[column_name], name="Box plot"),
        row=2, col=1
        )

    fig.update_layout(height=600, width=800, title_text=f"Distribution of '{column_name}' values")
    fig.show()

    return 

In [21]:
def plot_surface_impact(column_name):

    fig = make_subplots(rows=3, cols=1)

    variable_impact = pd.DataFrame()
    variable_impact['mean_impact'] = df.groupby(['loan_status'])[column_name].mean()
    variable_impact['max_impact'] = df.groupby(['loan_status'])[column_name].max()
    variable_impact['min_impact'] = df.groupby(['loan_status'])[column_name].min()

    fig.add_trace(
        go.Bar(y=variable_impact['mean_impact'], x=variable_impact.index, name="Impact to mean"),
        row=1, col=1
        )

    fig.add_trace(
        go.Bar(y=variable_impact['max_impact'], x=variable_impact.index, name="Impact to max"),
        row=2, col=1
        )
    
    fig.add_trace(
        go.Bar(y=variable_impact['min_impact'], x=variable_impact.index, name="Impact to min"),
        row=3, col=1
        )

    fig.update_layout(height=600, width=800, title_text=f"Impact of '{column_name}' on surface statsitics of 'loan_status' values")
    fig.show()

    return 

### Feature Plots

To visualize the data we can plot a number of selected variables to get impression of their distribution.

In [22]:
plot_distribution('loan_amnt')
plot_surface_impact('loan_amnt')

plot_distribution('funded_amnt')
plot_surface_impact('funded_amnt')

plot_distribution('emp_length')
plot_surface_impact('emp_length')

plot_surface_impact('term_in_months')

plot_distribution('int_rate')
plot_surface_impact('int_rate')


### Correlation Matrix

In [23]:
# draft correlation matrix
df_corr = df.corr() # Generate correlation matrix

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.columns,
        z = np.array(df_corr)
    )
)
fig.update_layout(height=1000, width=1000)
fig.show()